In [1]:
import sys
sys.path.append('../')
from utils import paths
from utils import utils

import warnings
from tqdm import tqdm
import pandas as pd
import numpy as np
import datetime
import xarray as xr
import time

In [34]:
### We want to create the dataset, given the Location(lat, lon):

def prepare_data(location):
    '''
    location: name of location, in lowercase, e.g. 'aberdeen'.
    '''
    lat, lon = GetCoordinates(location)
    # Retrieve data from specified location in the UK
    
    # Perform data preprocessing or cleaning
    
    # Save prepared data to desired location

def geo_coord_borders(lat, lon):
    """
    Get the borders of the window of interest for CMIP6 Native grid;
    To be passed down into extract_window function.
    ---
    lat (np.float): latitude of location
    lon (np.float): longitude of location
    ---
    Returns:
        ((lat_lw_bord,lat_up,bord),(lon_lw_bord,lon_up_bord)): (tuple(tuple(np.float)))
    """
    ### The CMIP6 N96 Native grid has   144 gridcells of latitude, i.e. 1.25deg/cell, and
    ###                                 192 gridcells of longitude, i.e. 1.875deg/cell.

    ### Make sure the longitude is in [-180,180], not [0,360]:
    if (lon - 180 >0):
        lon = lon - 360

    delta_lat = 1.25 * 2.5
    delta_lon = 1.875* 2.5

    lat_lw_bord = lat - delta_lat 
    lat_up_bord = lat + delta_lat

    lon_lw_bord = lon - delta_lon
    lon_up_bord = lon + delta_lon

    if ((lon_lw_bord >= 0)&(lon_up_bord > 0)):
        lon_lw_bord = lon - delta_lon
        lon_up_bord = lon + delta_lon
    elif ((lon_lw_bord < 0)&(lon_up_bord <= 0)):
        lon_lw_bord = lon - delta_lon + 360
        lon_up_bord = lon + delta_lon + 360
    elif ((lon_lw_bord < 0)&(lon_up_bord > 0)):
        lon_lw_bord = lon - delta_lon + 360
        lon_up_bord = lon + delta_lon
    else:
        print("Error: Something is wrong with the Longitude.")

    return ((lat_lw_bord,lat_up_bord),(lon_lw_bord,lon_up_bord))

#--------------------------------------------------------------------------------------------------
def extract_window(data,geo_coord_tuple,data_type = 'CMIP6'):
    """
    data_type (string): 'CMIP6' or 'ERA5'
    THIS FUNCTION FAILS FOR NEGATIVE LONGITUDES ---- FIX!
    """


    if data_type == 'CMIP6':

        lat_lw = geo_coord_tuple[0][0]
        lat_up = geo_coord_tuple[0][1]
        lon_lw = geo_coord_tuple[1][0]
        lon_up = geo_coord_tuple[1][1]

        data = data.sel(lat = slice(lat_lw,lat_up))

        ### Check if the window crosses the meridian:
        if ((lon_lw >180)&(lon_lw<360))&((lon_up>0)&(lon_up<180)):
            crop1 = data.sel(lon = slice(lon_lw,360))
            crop2 = data.sel(lon = slice(0,lon_up))
            data = xr.concat([crop1,crop2],dim = 'lon')
        else:
            data = data.sel(lon = slice(lon_lw,lon_up))
    elif data_type == 'ERA5':
        ### ERA5 data has longitude in [-180,180]
        lat_lw = geo_coord_tuple[0][1]
        lat_up = geo_coord_tuple[0][0]
        lon_lw = geo_coord_tuple[1][0]
        lon_up = geo_coord_tuple[1][1]

        data = data.sel(latitude = slice(lat_lw,lat_up))

        ### Check if the window crosses the meridian:
        if ((lon_lw >180)&(lon_lw<360))&((lon_up>0)&(lon_up<180)):
            crop1 = data.sel(longitude = slice(lon_lw-360,360))
            crop2 = data.sel(longitude = slice(0,lon_up))
            data = xr.concat([crop1,crop2],dim = 'longitude')
        elif ((lon_lw >180)&(lon_lw<360))&((lon_up>180)&(lon_up<360)):
            data = data.sel(longitude = slice(lon_lw-360,lon_up-360))
        else:
            data = data.sel(longitude = slice(lon_lw,lon_up))
    else:
        print('data_type not recognized.')

    return data



In [7]:
uk_haigh_meta = pd.read_csv(paths.Haigh_meta_path).drop(['Unnamed: 0','index'],axis=1)
uk_haigh_meta.head()

,location,latitude,longitude,file_name,geometry
0,aberdeen,57.14325,-2.07451,aberdeen-p038-uk-bodc,POINT (-2.07451 57.14325)
1,avonmouth,51.51089,-2.71497,avonmouth-p060-uk-bodc,POINT (-2.71497 51.51089)
2,bangor,54.66475,-5.66947,bangor-p662-uk-bodc,POINT (-5.66947 54.66475)
3,barmouth,52.71906,-4.04517,barmouth-p923-uk-bodc,POINT (-4.04517 52.71906)
4,bournemouth,50.71433,-1.87486,bournemouth-p988-uk-bodc,POINT (-1.87486 50.71433)


In [24]:
storms = pd.read_csv(paths.storms_path, skiprows = 2, header = 3)
water_level= pd.read_csv(paths.water_levels_path,skiprows = 2, header =3)

In [4]:
location = 'aberdeen'
lat, lon = utils.GetCoordinates(location)
flood_records = utils.ExtractFloods(location)
geo_borders = utils.GetGeoBorders(lat,lon)

In [11]:
def GetFormattedData(location):
    start = time.time()
    warnings.filterwarnings("ignore")
    ps_data, u_data, v_data, t_data, pr_data = Step1()
    ps_data, u_data, v_data, t_data, pr_data = Step2(ps_data, u_data, v_data, t_data, pr_data)
    ps_window, u_window, v_window, t_window, pr_window = Step3(ps_data, u_data, v_data, t_data, pr_data, location)
    df_ps, df_u, df_v, df_t, df_pr = Step4(ps_window, u_window, v_window, t_window, pr_window)
    df_ps, df_u, df_v, df_t, df_pr = Step5(df_ps, df_u, df_v, df_t, df_pr)
    df_ps, df_u, df_v, df_t, df_pr = Step6(df_ps, df_u, df_v, df_t, df_pr)
    df_merged = Step7(df_ps, df_u, df_v, df_t, df_pr)
    final_df = Step8(df_merged)
    end = time.time()
    print("Time taken to get the data: ", end-start)
    return final_df

def Step1():
    # open datasets:
    start = time.time()
    ps_data = xr.open_dataset(paths.era5_79_18_ps)
    u_data = xr.open_dataset(paths.era5_79_18_u)
    v_data = xr.open_dataset(paths.era5_79_18_v)
    t_data = xr.open_dataset(paths.era5_79_18_t)
    pr_data = xr.open_dataset(paths.era5_79_18_pr)
    end = time.time()
    print("Time taken to open datasets: ", end-start, ' ; Step 1/8')
    return ps_data, u_data, v_data, t_data, pr_data

def Step2(ps_data, u_data, v_data, t_data, pr_data):
    start = time.time()
    # remove leap years:
    ps_data = ps_data.sel({'time': ~((ps_data['time'].dt.month == 2) & (ps_data['time'].dt.day == 29))})
    u_data = u_data.sel({'time': ~((u_data['time'].dt.month == 2) & (u_data['time'].dt.day == 29))})
    v_data = v_data.sel({'time': ~((v_data['time'].dt.month == 2) & (v_data['time'].dt.day == 29))})
    t_data = t_data.sel({'time': ~((t_data['time'].dt.month == 2) & (t_data['time'].dt.day == 29))})
    pr_data = pr_data.sel({'time': ~((pr_data['time'].dt.month == 2) & (pr_data['time'].dt.day == 29))})
    end = time.time()
    print("Time taken to remove leap years: ", end-start, ' ; Step 2/8')
    return ps_data, u_data, v_data, t_data, pr_data

def Step3(ps_data, u_data, v_data, t_data, pr_data, location):
    start = time.time()
    warnings.filterwarnings("ignore")
    lat, lon = utils.GetCoordinates(location)
    geo_borders = utils.GetGeoBorders(lat,lon)
    # Get the window around the location:
    ps_window = utils.ExtractWindow(ps_data,geo_borders,data_type='ERA5')
    u_window = utils.ExtractWindow(u_data,geo_borders,data_type='ERA5')    
    v_window = utils.ExtractWindow(v_data,geo_borders,data_type='ERA5')
    t_window = utils.ExtractWindow(t_data,geo_borders,data_type='ERA5')
    pr_window = utils.ExtractWindow(pr_data,geo_borders,data_type='ERA5')
    end = time.time()
    print("Time taken to extract window: ", end-start, ' ; Step 3/8')
    return ps_window, u_window, v_window, t_window, pr_window

def Step4(ps_window, u_window, v_window, t_window, pr_window):
    start = time.time()
    # Get the dataframes:
    df_ps = ps_window.sp.to_dataframe().reset_index()
    df_u = u_window.u10.to_dataframe().reset_index()
    df_v = v_window.v10.to_dataframe().reset_index()
    df_t = t_window.t2m.to_dataframe().reset_index()
    df_pr = pr_window.tp.to_dataframe().reset_index()
    end = time.time()
    print("Time taken to convert to dataframe: ", end-start, ' ; Step 4/8')
    return df_ps, df_u, df_v, df_t, df_pr

def Step5(df_ps, df_u, df_v, df_t, df_pr):
    start = time.time()
    df_ps['t'] = df_ps.time.apply(lambda x: utils.date_to_fractional_years_resample3hrs(x))
    df_u['t'] = df_ps['t']
    df_v['t'] = df_ps['t']
    df_t['t'] = df_ps['t']
    df_pr['t'] = df_ps['t']
    end = time.time()
    print("Time taken to convert to fractional years: ", end-start, ' ; Step 5/8')
    return df_ps, df_u, df_v, df_t, df_pr

def Step6(df_ps, df_u, df_v, df_t, df_pr):
    start = time.time()
    df_u.drop(columns=['time'],inplace=True)
    df_v.drop(columns=['time'],inplace=True)
    df_t.drop(columns=['time'],inplace=True)
    df_ps.drop(columns=['time'],inplace=True)
    df_pr.drop(columns=['time'],inplace=True)
    end = time.time()
    print("Time taken to drop time column: ", end-start, ' ; Step 6/8')
    return df_ps, df_u, df_v, df_t, df_pr

def Step7(df_ps, df_u, df_v, df_t, df_pr):
    start = time.time()
    df_merged = df_u.merge(df_v,on=['t','latitude','longitude'],how='inner')
    df_merged = df_merged.merge(df_t,on=['t','latitude','longitude'],how='inner')
    df_merged = df_merged.merge(df_ps,on=['t','latitude','longitude'],how='inner')
    df_merged = df_merged.merge(df_pr,on=['t','latitude','longitude'],how='inner')
    end = time.time()
    print("Time taken to merge dataframes: ", end-start, ' ; Step 7/8')
    return df_merged

def Step8(df_merged):
    start = time.time()
    lat_array= np.unique(df_merged['latitude'])
    lon_array= np.unique(df_merged['longitude'])
    lon_array[lon_array > 180] = lon_array[lon_array > 180] - 360
    lon_array.sort()

    for i in tqdm(range(25)):
        LAT = lat_array[i]
        for j in range(25):
            LON = lon_array[j]
            # if LON<0: LON = LON + 360

            A = df_merged[df_merged['latitude'] == LAT]
            B = A[A['longitude'] == LON]
            B = B.drop(columns = ['latitude','longitude'])#, inplace = True)
            B = B.rename(columns= {'u10':'u_'+str(i+1)+'_'+str(j+1),
                                'v10':'v_'+str(i+1)+'_'+str(j+1),
                                't2m':'T_'+str(i+1)+'_'+str(j+1),
                                'sp':'P_'+str(i+1)+'_'+str(j+1),
                                'tp':'pr_'+str(i+1)+'_'+str(j+1)})
            if i==0 and j == 0:
                final_df = B
            else:
                final_df = final_df.merge(B,on='t',how='inner')
    end = time.time()
    print("Time taken to create the grid-cell-wise features: ", end-start, ' ; Step 8/8')
    return final_df

In [13]:
final_df_aberdeen = GetFormattedData('aberdeen')

Time taken to open datasets:  0.021620988845825195  ; Step 1/8
Time taken to remove leap years:  0.03525686264038086  ; Step 2/8
Time taken to extract window:  0.002850770950317383  ; Step 3/8
Time taken to convert to dataframe:  82.55139708518982  ; Step 4/8
Time taken to convert to fractional years:  83.929270029068  ; Step 5/8
Time taken to drop time column:  3.0039799213409424  ; Step 6/8
Time taken to merge dataframes:  107.78338122367859  ; Step 7/8


100%|██████████| 25/25 [03:52<00:00,  9.29s/it]

Time taken to create the grid-cell-wise features:  234.99720978736877  ; Step 8/8
Time taken to get the data:  512.3287978172302


In [10]:
final_df_aberdeen

,u_1_1,t,v_1_1,T_1_1,P_1_1,pr_1_1,u_1_2,v_1_2,T_1_2,P_1_2,...,u_25_24,v_25_24,T_25_24,P_25_24,pr_25_24,u_25_25,v_25_25,T_25_25,P_25_25,pr_25_25
0,-4.287253,1979.000000,-6.412828,272.940887,101650.906250,2.506189e-05,-4.466002,-6.803702,272.587158,101359.156250,...,-3.549765,8.276287,270.445709,101598.859375,5.457737e-05,-3.804275,7.801084,270.116364,101628.304688,6.126054e-05
1,-2.441757,1979.000342,-5.705153,273.148224,101902.937500,1.115724e-06,-2.774396,-5.909136,272.749207,101616.664062,...,-2.577890,7.918460,271.113068,101617.343750,8.019432e-05,-2.803990,7.423885,270.693115,101655.703125,7.462688e-05
2,0.916595,1979.000685,-4.984941,273.543762,102043.328125,1.115724e-06,0.338916,-5.134226,273.022766,101759.109375,...,-4.209124,8.112188,271.754272,101609.125000,2.010427e-04,-4.554784,7.584565,271.318665,101659.125000,1.620594e-04
3,5.835155,1979.001027,-0.179356,274.211121,102134.421875,2.172031e-05,5.710859,-0.468808,274.435883,101840.609375,...,-5.861665,8.931541,272.179443,101703.640625,1.576040e-04,-5.895995,8.236401,271.844879,101754.320312,9.300373e-05
4,9.897849,1979.001370,0.128329,274.754761,102215.234375,5.587935e-07,10.333476,-0.077934,275.145081,101920.054688,...,-6.602705,8.904192,272.390259,101759.109375,1.175068e-04,-6.500900,8.066605,272.109741,101805.000000,5.791895e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116795,6.373770,2018.998288,5.895715,282.783936,103324.718750,3.285892e-05,7.295926,5.949275,282.698547,103061.726562,...,12.554226,8.367453,283.008728,101449.554688,1.024716e-04,12.355353,8.379988,282.968628,101468.046875,1.080409e-04
116796,6.770332,2018.998630,4.397175,282.925079,103280.882812,4.455447e-05,7.543334,4.508853,282.785675,103017.890625,...,14.102596,8.057487,283.503571,101146.156250,2.077240e-04,14.132190,7.940112,283.472198,101157.117188,2.405811e-04
116797,7.294742,2018.998973,3.880948,283.198639,103248.695312,4.678033e-05,8.143503,3.962997,283.031372,102978.859375,...,14.539407,7.477445,283.372894,100822.906250,3.330261e-04,14.376046,7.803363,283.372894,100836.601562,2.517197e-04
116798,7.764698,2018.999315,3.455886,283.313660,103246.640625,6.685033e-06,8.889278,3.495771,283.172516,102969.265625,...,17.629044,0.628602,281.562500,100663.328125,1.862645e-09,17.398209,0.617207,281.572937,100662.648438,1.862645e-09
